In [1]:
import pandas as pd
import numpy as np
import time
import os
import sys
import utils 
import itertools
import random as rand
import math
import copy
sys.path.append('../')
sys.path.insert(1, '../exp')

# sklearn
from sklearn.model_selection import cross_validate, train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn.feature_selection import mutual_info_classif
from sklearn.utils import compute_class_weight

import xgboost as xgb

#PostgreSQL
import psycopg2
from sqlalchemy import create_engine

# sqlgen
import sqlgen
from sqlgen import study, trial
from sqlgen.samplers import tpe, random

# graph
import matplotlib.pyplot as plt
import plotly

# featuretools
import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

## Step 1. Load data

In [2]:
path = "../exp_data/Tmall/"
train_data = pd.read_csv(os.path.join(path, "train_data.csv"))
test_data = pd.read_csv(os.path.join(path, "test_data.csv"))
user_log = pd.read_csv(os.path.join(path, "user_log.csv"))

train_data = train_data.drop(train_data.columns[0], axis=1)
test_data = test_data.drop(test_data.columns[0], axis=1)
user_log = user_log.drop(user_log.columns[0], axis=1)
# data = pd.concat([train_data, test_data])
print(len(user_log))
print("Train true label:", len(train_data[train_data['label']==1]), 'Train false label:', len(train_data[train_data['label']==0]))
print("Test true label:", len(test_data[test_data['label']==1]), 'Test false label:', len(test_data[test_data['label']==0]))
train_data.head()

345699
Train true label: 799 Train false label: 801
Test true label: 208 Test false label: 192


,user_id,merchant_id,age_range,gender,item_id,cat_id,brand_id,action_type,label
0,213820,2066,2,0,5,6,7,0,0
1,195,2286,5,0,16,8,5,1,1
2,318509,4143,6,0,8,6,11,0,0
3,323850,361,6,0,6,0,13,0,1
4,380935,3191,3,0,4,8,12,0,1


## Step 2. Store table in PostragSQL
Store train_data to database

In [ ]:
engine = utils.store_tmall(user_log)

## Step 3. Feature Generation & Machine Learning

In [ ]:
train_data.head()

In [ ]:
valid_score, clf = utils.model_tmall(train_data)
test_pred = clf.predict(test_data.drop(columns='label', axis=1))
test_score = metrics.roc_auc_score(test_data['label'], test_pred)
print(valid_score, test_score)

In [ ]:
global copy_train_data, copy_test_data, train_data, test_data
global trail_auc_store, trail_mi_store, train_auc_store

trail_auc_store = []
trail_mi_store = []
train_auc_store = []
copy_train_data = train_data
copy_test_data = test_data

global random_args_log, tpe_args_log
global random_user_train_log, tpe_user_train_log

random_args_log = {}
tpe_args_log = {}
random_user_train_log = {}
tpe_user_train_log = {}

global eva_global, seed_global

In [ ]:
# res_x is to store top features 
global res_x 
res_x = list()

aggregation = ['SUM', 'MIN', 'MAX', 'COUNT', 'AVG'] 
m_attr = ['merchant_id', 'item_id', 'brand_id', 'cat_id']
categorical = ['gender', 'age_range', 'action_type']
numerical = ['time_stamp']
attributes = categorical + numerical

In [ ]:
def update_user_train(args, train_data):  
    agg, m, lb_day, ub_day, w_cat_value_0, w_cat_value_1, w_cat_value_2 = args 
    agg = aggregation[agg]
    m = m_attr[m]
    
    w_cat_value_temp = [w_cat_value_0, w_cat_value_1, w_cat_value_2] #, w_cat_value_3]
    w_cat = []
    w_cat_value = []

    for i in range(len(categorical)):
        if w_cat_value_temp[i] != -1:
            w_cat.append(categorical[i])
            w_cat_value.append(w_cat_value_temp[i])
            
    sql_output = utils.generate_feature_in_small_space_tmall(engine, agg, m, lb_day, ub_day, w_cat, w_cat_value) 
    w_cat_str = "_".join(str(w_cat[x])+'='+str(w_cat_value[x]) for x in range(len(w_cat)))
    new_feature = pd.DataFrame(sql_output, columns = ['user_id', agg+'('+m+')_'+w_cat_str+'_'+'lb_day='+str(lb_day)+"_ub_day="+str(ub_day)]) #+'='+g_cat
    new_feature = new_feature.astype('float')
    new_train_data = train_data.merge(new_feature, how='left')
    
    return new_train_data, args

In [ ]:
# define an objective function
def objective(trial):
    agg = trial.suggest_categorical('agg', np.array([i for i in range(len(aggregation))]))
    m = trial.suggest_categorical('m', np.array([i for i in range(len(m_attr))]))  
    lb_day = trial.suggest_categorical('lb_day', sorted_time[0: math.ceil(len(sorted_time)/2)])
    ub_day = trial.suggest_categorical('ub_day', sorted_time[math.ceil(len(sorted_time)/2)+1: -1])
    w_cat_value_0 = trial.suggest_categorical('w_cat_value_0', np.append(user_log[categorical[0]].unique(), [-1], 0))
    w_cat_value_1 = trial.suggest_categorical('w_cat_value_1', np.append(user_log[categorical[1]].unique(), [-1], 0))
    w_cat_value_2 = trial.suggest_categorical('w_cat_value_2', np.append(user_log[categorical[2]].unique(), [-1], 0))

    args = [agg, m, lb_day, ub_day, w_cat_value_0, w_cat_value_1, w_cat_value_2] #, w_cat_value_3]
    new_train_data, args = update_user_train(args, train_data)
    score, _ = utils.model_tmall(new_train_data)
    
    for x in res_x:
        if args == x:
            score -= 0.2
            break
        
    print(args, " auc: ", score)
    return score

In [ ]:
# define a NEW function with mutual information

def mi(trial):
    agg = trial.suggest_categorical('agg', np.array([i for i in range(len(aggregation))]))
    m = trial.suggest_categorical('m', np.array([i for i in range(len(m_attr))]))  
    lb_day = trial.suggest_categorical('lb_day', sorted_time[0: math.ceil(len(sorted_time)/2)])
    ub_day = trial.suggest_categorical('ub_day', sorted_time[math.ceil(len(sorted_time)/2)+1: -1])
    w_cat_value_0 = trial.suggest_categorical('w_cat_value_0', np.append(user_log[categorical[0]].unique(), [-1], 0))
    w_cat_value_1 = trial.suggest_categorical('w_cat_value_1', np.append(user_log[categorical[1]].unique(), [-1], 0))
    w_cat_value_2 = trial.suggest_categorical('w_cat_value_2', np.append(user_log[categorical[2]].unique(), [-1], 0))

    args = [agg, m, lb_day, ub_day, w_cat_value_0, w_cat_value_1, w_cat_value_2] #, w_cat_value_3]
    new_train_data, args = update_user_train(args, train_data)

    mi_matrix = mutual_info_classif(new_train_data.fillna(0), new_train_data['label'], random_state=0)
    mi_score = mi_matrix[-1]
    
    for x in res_x:
        if args == x:
            mi_score = 0
            break
            
    print(args, "MI:", mi_score)
    return mi_score

In [ ]:
# Random sampling 
def evaluate(trial_arr, mi_log, train_size):
    x_features = pd.DataFrame()
    x_features['index'] = range(0, len(mi_log))
    x_features['mi'] = mi_log
    x_features['agg'] = [trial_arr[i]['agg'] for i in range(len(trial_arr))]
    x_features['m'] = [trial_arr[i]['m'] for i in range(len(trial_arr))]
    x_features['lb_day'] = [trial_arr[i]['lb_day'] for i in range(len(trial_arr))]
    x_features['ub_day'] = [trial_arr[i]['ub_day'] for i in range(len(trial_arr))]
    x_features['w_cat_value_0'] = [trial_arr[i]['w_cat_value_0'] for i in range(len(trial_arr))]
    x_features['w_cat_value_1'] = [trial_arr[i]['w_cat_value_1'] for i in range(len(trial_arr))]
    x_features['w_cat_value_2'] = [trial_arr[i]['w_cat_value_2'] for i in range(len(trial_arr))]

    print(x_features.shape)
    x_train, x_test = train_test_split(x_features, train_size=200, random_state=0)
    
    y_acc = []
    for index, feature in x_train.iterrows():
        args = [int(feature['agg']),
               int(feature['m']),
               int(feature['lb_day']),
               int(feature['ub_day']),
               int(feature['w_cat_value_0']),
               int(feature['w_cat_value_1']),
               int(feature['w_cat_value_2'])] 
        new_train_data, args = update_user_train(args, train_data)
        
        y_acc.append(utils.model_tmall(new_train_data)[0])

    y_train = pd.DataFrame()
    y_train['index'] = x_train['index']
    y_train['label'] = y_acc

    global clf
    clf =  RandomForestRegressor(random_state=0) #LinearRegression()
    clf.fit(x_train[['mi']], y_train['label'])
    
    global estimated_accuracy
    estimated_accuracy = clf.predict(x_features[['mi']])
    predict_y = pd.DataFrame()
    predict_y['index'] = range(0, len(mi_log))
    predict_y['label'] = estimated_accuracy
    
    for index, row in y_train.iterrows():
        predict_y.loc[predict_y['index'] == row['index'], 'label'] = row['label']
    
    best_trail_index = y_train.nlargest(1, ['label']).index[0]
    train_auc_store.append(predict_y['label'].to_numpy())
    return predict_y['label'].to_numpy(), best_trail_index

In [ ]:
top = 5

n_calls = [1500]
random_state_arr = [0] 

for n_call in n_calls:
    tpe_args_log[n_call] = {}
    tpe_user_train_log[n_call] = {}
    for seed in random_state_arr:
        tpe_args_log[n_call][seed] = []


# SQLGEN - Opt3

In [ ]:
eva_store_optuna = []

start = time.time()
for eva in n_calls:
    print("trails:", eva)
    eva_global = eva
    seed_store = []
    for seed in random_state_arr:
        seed_global = seed
        global train_data, res_x 
        res_optuna_lst = []
        res_optuna_fun = []
        res_x = list()
        train_data = copy_train_data
        print("seed:", seed)

        for i in range(top):
            tpe_result = study.create_study(
                direction="maximize", 
                sampler=tpe.TPESampler(n_startup_trials=20, seed=seed)) #
            tpe_result.optimize(objective, n_trials=eva, 
                                mi_initializer=mi, evaluate=evaluate, mi_init_trails=1000, train_size=100)
            
            best_trial = []
            for key, value in tpe_result.best_trial.params.items():
                best_trial.append(value)
            res_x.append(best_trial)

            train_data, args = update_user_train(best_trial, train_data)
            res_optuna_lst.append(tpe_result.best_trial.params)
            res_optuna_fun.append(tpe_result.best_value)

            print(tpe_result.best_trial.params, tpe_result.best_value)
            print(train_data.shape)
            tpe_args_log[eva][seed].append(args)
        seed_store.append(res_optuna_fun)
        tpe_user_train_log[eva][seed] = train_data
        
    eva_store_optuna.append(seed_store)
end = time.time()
print("Time:", end-start)

# Evaluate Test ROC-AUC

In [ ]:
def model_tmall(user_train):
    y = user_train['label']
    y = y.to_frame()
    X = user_train.drop(['label'], axis=1)
    X = X.fillna(0)

    clf = xgb.XGBRegressor(random_state=0)
    auc = cross_validate(clf, X, y, cv=5,scoring=('roc_auc'), return_train_score=True, n_jobs=-1, return_estimator=True)
    valid_auc = auc['test_score'].mean()
    train_auc = auc['train_score'].mean()

    return train_auc, valid_auc, auc['estimator'][0]


In [ ]:
test_auc = {}
train_auc = {}
valid_auc = {}
for n_calls in tpe_args_log:
    for seed in tpe_args_log[n_calls]:
        test_auc[seed] = []
        train_auc[seed] = []
        valid_auc[seed] = []
        train_data = copy_train_data
        test_data = copy_test_data
        
        # Default AUC without new features
        train, valid, clf = model_tmall(train_data)
        train_auc[seed].append(train)
        valid_auc[seed].append(valid)
        test_pred = clf.predict(test_data.drop(columns='label', axis=1))
        test_auc[seed].append(metrics.roc_auc_score(test_data['label'], test_pred))
        for args in tpe_args_log[n_calls][seed]:
            test_data, config = update_user_train(args, test_data)
            train_data, config = update_user_train(args, train_data)
            train, valid, clf = model_tmall(train_data)
            train_auc[seed].append(train)
            valid_auc[seed].append(valid)
            
            test_data = test_data.fillna(0)
            test_pred = clf.predict(test_data.drop(columns='label', axis=1))
            test_auc[seed].append(metrics.roc_auc_score(test_data['label'], test_pred))
train_auc, valid_auc, test_auc